In [5]:
# Water Quality Analysis System: LEWAS Lab
# Anomaly Detection for Dissolved Oxygen Data

## 1. Setup and Configuration
# First, we'll import all necessary libraries and set up our environment:

# Data manipulation and analysis
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Statistical analysis
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Setup complete!")


## 2. Data Loading and Initial Processing
# load and process the water_DO.csv file:

def load_do_data(file_path='water_DO.csv'):
    """
    Load and process LEWAS Lab dissolved oxygen data with proper datetime parsing.
    """
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
    df = df.sort_values('datetime')
    
    print("\nDataset Overview:")
    print(f"Time Range: {df['datetime'].min()} to {df['datetime'].max()}")
    print(f"Number of readings: {len(df)}")
    print(f"Sampling frequency: {df['datetime'].diff().mean()}")
    print("\nBasic statistics of DO values:")
    print(df['value'].describe())
    
    return df

# Load the data
do_data = load_do_data('water_DO.csv')


## 3. Data Quality Assessment
# Now check the quality of our data:


def assess_data_quality(df):
    """
    Perform comprehensive data quality assessment.
    """
    print("\nData Quality Assessment:")
    
    missing = df.isnull().sum()
    print("\nMissing Values:")
    print(missing[missing > 0])
    
    print("\nValue Ranges:")
    print(f"DO Range: {df['value'].min():.2f} to {df['value'].max():.2f} mg/L")
    
    time_gaps = df['datetime'].diff()
    print("\nTime Gaps:")
    print(f"Minimum gap: {time_gaps.min()}")
    print(f"Maximum gap: {time_gaps.max()}")
    print(f"Mean gap: {time_gaps.mean()}")
    
    duplicates = df.duplicated('datetime').sum()
    print(f"\nDuplicate timestamps: {duplicates}")
    
    return {
        'missing_values': missing,
        'time_gaps': time_gaps,
        'duplicates': duplicates
    }

# Assess data quality
quality_metrics = assess_data_quality(do_data)


## 4. Data Visualization
# create comprehensive visualizations:

def create_visualizations(df):
    """
    Create comprehensive visualizations of the DO data using matplotlib instead of plotly.
    """
    df = df.copy()
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['hour'] = df['datetime'].dt.hour
    df['month'] = df['datetime'].dt.month
    df['year'] = df['datetime'].dt.year
    
    fig = plt.figure(figsize=(20, 15))
    
    plt.subplot(3, 2, 1)
    plt.plot(df['datetime'], df['value'], 'b-', alpha=0.5)
    plt.title('Dissolved Oxygen Over Time')
    plt.xlabel('Date')
    plt.ylabel('DO (mg/L)')
    plt.xticks(rotation=45)
    
    plt.subplot(3, 2, 2)
    sns.histplot(data=df, x='value', bins=50)
    plt.title('Distribution of DO Values')
    plt.xlabel('DO (mg/L)')
    
    plt.subplot(3, 2, 3)
    sns.boxplot(data=df, x='month', y='value')
    plt.title('DO Values by Month')
    plt.xlabel('Month')
    plt.ylabel('DO (mg/L)')
    
    plt.subplot(3, 2, 4)
    sns.boxplot(data=df, x='hour', y='value')
    plt.title('DO Values by Hour')
    plt.xlabel('Hour of Day')
    plt.ylabel('DO (mg/L)')
    
    plt.subplot(3, 2, 5)
    sns.boxplot(data=df, x='year', y='value')
    plt.title('DO Values by Year')
    plt.xlabel('Year')
    plt.ylabel('DO (mg/L)')
    
    plt.subplot(3, 2, 6)
    time_gaps = df['datetime'].diff().dt.total_seconds() / 60
    sns.histplot(time_gaps[time_gaps < time_gaps.quantile(0.95)])
    plt.title('Distribution of Time Gaps')
    plt.xlabel('Gap (minutes)')
    plt.ylabel('Count')
    
    plt.tight_layout()
    plt.savefig('do_analysis.png')  # Save the plot instead of showing it
    plt.close()

# Create visualizations


## 5. Data Preprocessing
# Now prepare the data for anomaly detection:


def preprocess_data(df, resample_freq='1H'):
    """
    Preprocess the DO data for anomaly detection.
    """
    df_copy = df.copy()
    df_copy['datetime'] = df_copy['datetime'].dt.tz_localize(None)
    df_copy.set_index('datetime', inplace=True)
    
    df_resampled = df_copy['value'].resample(resample_freq).mean()
    df_resampled = df_resampled.interpolate(method='linear')
    
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(df_resampled.values.reshape(-1, 1))
    
    processed_df = pd.DataFrame(
        scaled_values, 
        index=df_resampled.index, 
        columns=['scaled_value']
    )
    processed_df['original_value'] = df_resampled.values
    
    return processed_df, scaler

# Preprocess the data

    # Create visualizations
create_visualizations(do_data)
    
    # Preprocess data for anomaly detection
processed_data, scaler = preprocess_data(do_data)
    
print("\nData preprocessing completed successfully!")



## 6. Anomaly Detection
# implement both statistical and LSTM-based anomaly detection:

def detect_anomalies(df, window_size=24, z_threshold=3):
    """
    Detect anomalies using statistical methods.
    """
    rolling_mean = df['scaled_value'].rolling(window=window_size, center=True).mean()
    rolling_std = df['scaled_value'].rolling(window=window_size, center=True).std()
    
    z_scores = abs((df['scaled_value'] - rolling_mean) / rolling_std)
    
    df['is_anomaly'] = z_scores > z_threshold
    df['z_score'] = z_scores
    
    df['anomaly_type'] = 'normal'
    df.loc[z_scores > z_threshold * 2, 'anomaly_type'] = 'sensor_fault'
    df.loc[(z_scores > z_threshold) & (z_scores <= z_threshold * 2), 
           'anomaly_type'] = 'environmental'
    
    return df

# Detect anomalies
data_with_anomalies = detect_anomalies(processed_data)

## 7. Visualize Results
# create interactive visualizations of the anomalies:

def plot_anomalies(df):
    """
    Create visualization of anomaly detection results using matplotlib instead of plotly.
    """
    plt.figure(figsize=(15, 8))
    
    # Plot original data
    plt.plot(df.index, df['original_value'], 'b-', label='Original Data', alpha=0.5)
    
    # Plot sensor faults
    sensor_faults = df[df['anomaly_type'] == 'sensor_fault']
    plt.scatter(sensor_faults.index, sensor_faults['original_value'], 
               color='red', marker='x', s=100, label='Sensor Faults')
    
    # Plot environmental anomalies
    env_anomalies = df[df['anomaly_type'] == 'environmental']
    plt.scatter(env_anomalies.index, env_anomalies['original_value'],
               color='green', marker='o', s=100, label='Environmental Anomalies')
    
    plt.title("Dissolved Oxygen Anomaly Detection Results")
    plt.xlabel("DateTime")
    plt.ylabel("Dissolved Oxygen (mg/L)")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    plt.savefig('anomalies.png')  # Save the plot instead of showing it
    plt.close()

# Plot results
plot_anomalies(data_with_anomalies)


## 8. Analysis Summary
# create a summary of our findings:


def create_summary(df):
    """
    Create a summary of the anomaly detection results.
    """
    print("\nAnomaly Detection Summary:")
    print(f"Total observations: {len(df)}")
    print(f"Total anomalies detected: {df['is_anomaly'].sum()}")
    print(f"Sensor faults: {(df['anomaly_type'] == 'sensor_fault').sum()}")
    print(f"Environmental anomalies: {(df['anomaly_type'] == 'environmental').sum()}")
    
    print("\nTemporal Distribution of Anomalies:")
    df['month'] = df.index.month
    monthly_anomalies = df[df['is_anomaly']].groupby('month').size()
    print(monthly_anomalies)

# Create summary
create_summary(data_with_anomalies)

Setup complete!

Dataset Overview:
Time Range: 2015-02-21 21:28:19+00:00 to 2018-03-22 16:39:15+00:00
Number of readings: 456444
Sampling frequency: 0 days 00:03:32.913016521

Basic statistics of DO values:
count    456444.000000
mean          9.123842
std           2.074245
min           0.000000
25%           8.170000
50%           8.920000
75%           9.980000
max          41.470000
Name: value, dtype: float64

Data Quality Assessment:

Missing Values:
sensor_id    452157
stderr       456444
dtype: int64

Value Ranges:
DO Range: 0.00 to 41.47 mg/L

Time Gaps:
Minimum gap: 0 days 00:00:00
Maximum gap: 124 days 22:51:25
Mean gap: 0 days 00:03:32.913016521

Duplicate timestamps: 343

Data preprocessing completed successfully!

Anomaly Detection Summary:
Total observations: 26996
Total anomalies detected: 13
Sensor faults: 0
Environmental anomalies: 13

Temporal Distribution of Anomalies:
month
2     1
5     1
7     1
8     7
9     1
10    1
11    1
dtype: int64
